In [1]:
%reset -f

In [2]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps
import json
# Import API key
from config import g_key

In [3]:
#Create a DataFrame for the public record data
public_record_df = pd.read_csv("Public_Record_Official_11_25_22.csv")
public_record_df.tail()

,MLS Number,PropertyAddressFormatted,PropertyCityState,Zipcode,Zip4,CarrierRoute,PropDoNotMail,OwnerNames,OwnerLastName,OwnerFirstName,...,Bedrooms,Exterior,BsmtDesc,FireplaceTotal,GrgType,PoolType,HeatDelivery,YearBuilt,YearRemod,CountyBldgDesc
5571,DENC520114,101 Meriden,"Hockessin, DE",19707,1704.0,R004,N,Michael J N Generations,Generations,Michael J N,...,3.0,"Frame, Siding - Wood, Stone",NaN,1.0,NaN,NaN,Hot/Warm Air,1958.0,0.0,NaN
5572,DENC520392,417 Snuff Mill Hill,"Hockessin, DE",19707,9638.0,R002,N,Jacqueline G Truluck & Breck L Robinson,Truluck,Jacqueline G,...,4.0,"Siding - Wood, Frame",Finished,2.0,Att/BuiltIn/Bsmt,NaN,Heat Pump,1973.0,0.0,NaN
5573,DENC2014038,400 Wilson,"Hockessin, DE",19707,9207.0,R001,N,Kevin K Thomas,Thomas,Kevin K,...,4.0,"Brick, Aluminum, Vinyl",Finished,1.0,Att/BuiltIn/Bsmt,NaN,Hot/Warm Air,1989.0,0.0,NaN
5574,DENC2005484,918 Old Public,"Hockessin, DE",19707,9679.0,R011,N,Lawrence P Murphy,Murphy,Lawrence P,...,4.0,"Aluminum, Vinyl, Brick",NaN,1.0,Att/BuiltIn/Bsmt,NaN,Hot/Warm Air,1992.0,0.0,NaN
5575,DENC526982,124 Croom Mills Drive,"Hockessin, DE",19707,8924.0,R013,N,Lee S & Dorel K Peplinski,Peplinski,Lee S,...,2.0,"Vinyl, Brick, Aluminum",NaN,0.0,Att/BuiltIn/Bsmt,NaN,Hot/Warm Air,2002.0,0.0,NaN


In [4]:

# Concatenate property address for the API pull
public_record_df["address_new"] = public_record_df["PropertyAddressFormatted"].astype(str) +" "+ public_record_df["PropertyCityState"]


In [5]:
# Create a list of property addresses using the concatenated format
pr_address_new_list= public_record_df["address_new"]
pr_address_new_list.head()


0    2615 Pecksniff  Wilmington, DE
1     4938 S Tupelo  Wilmington, DE
2       15 Kristina  Wilmington, DE
3    3251 Champions  Wilmington, DE
4    3706 Lafayette  Wilmington, DE
Name: address_new, dtype: object

In [6]:
# Establish a Lat/Lng Dataframe
prop_address_new_df = pd.DataFrame([{"address_new": " ", "lat": 0, "lng": 0}])
prop_address_new_df

,address_new,lat,lng
0,,0,0


In [7]:
# Loop through the rows and pull the APIs; print out any errors
for row in pr_address_new_list:
    params = {
    "key": g_key,
    "address": row}
        # base URL: 
    base_url = "https://maps.googleapis.com/maps/api/geocode/json?"
    # Make request and get the JSON data from the search.
    try:
        prop_address_json = requests.get(base_url, params=params).json()
    #print(prop_address_json["results"][0]["geometry"]['location']['lat'])
        lat = prop_address_json["results"][0]["geometry"]['location']['lat']
        lng = prop_address_json["results"][0]["geometry"]['location']['lng']
        prop_address_new_df = prop_address_new_df.append(
              [{"address_new": row, 
             "lat": lat,
             "lng": lng}],
             ignore_index=True)
    except:
        print("error on " + row)

error on 24 Yearsley  Wilmington, DE
error on 302 Park  Wilmington, DE
error on 302 Delaware  Newport, DE
error on 302 N Lake  Wilmington, DE


In [8]:
# Check the dataframe
prop_address_new_df.head()

,address_new,lat,lng
0,,0.000000,0.000000
1,"2615 Pecksniff Wilmington, DE",39.734608,-75.661402
2,"4938 S Tupelo Wilmington, DE",39.744655,-75.548391
3,"15 Kristina Wilmington, DE",39.744655,-75.548391
4,"3251 Champions Wilmington, DE",39.729664,-75.698656


In [9]:
# # Drop the first row of the dataframe which is now a duplicate
prop_address_new_df = prop_address_new_df.drop(index=0)
prop_address_new_df.head()

,address_new,lat,lng
1,"2615 Pecksniff Wilmington, DE",39.734608,-75.661402
2,"4938 S Tupelo Wilmington, DE",39.744655,-75.548391
3,"15 Kristina Wilmington, DE",39.744655,-75.548391
4,"3251 Champions Wilmington, DE",39.729664,-75.698656
5,"3706 Lafayette Wilmington, DE",39.727965,-75.634152


In [10]:
# Create a separate dataframe with the MLS key and the address_new
prop_MLS_address = public_record_df[["MLS Number","address_new"]]
prop_MLS_address.head()

,MLS Number,address_new
0,DENC518086,"2615 Pecksniff Wilmington, DE"
1,DENC518982,"4938 S Tupelo Wilmington, DE"
2,DENC512992,"15 Kristina Wilmington, DE"
3,DENC512104,"3251 Champions Wilmington, DE"
4,DENC503480,"3706 Lafayette Wilmington, DE"


In [11]:
# Merge the dataframes so that there are the MLS number, address_new and lat/lng in one dataframe
lat_lng_df = pd.merge(prop_MLS_address, prop_address_new_df, on="address_new", how="outer")
lat_lng_df.tail()

,MLS Number,address_new,lat,lng
5745,DENC520114,"101 Meriden Hockessin, DE",39.763823,-75.652016
5746,DENC520392,"417 Snuff Mill Hill Hockessin, DE",39.809390,-75.667333
5747,DENC2014038,"400 Wilson Hockessin, DE",39.780478,-75.717182
5748,DENC2005484,"918 Old Public Hockessin, DE",39.795708,-75.688011
5749,DENC526982,"124 Croom Mills Drive Hockessin, DE",39.755896,-75.704279


In [12]:
# Write the dataframe as a csv
lat_lng_df.to_csv("lat_lng_full.csv", index=False)